In [3]:
# ==============================================================================
# Phase 1: Setup and Dataset Preparation
# ==============================================================================

print("--- Phase 1: Setup and Dataset Preparation ---")

# Step 1: Install Libraries
# --------------------------
print("\nStep 1: Installing Libraries...")
!pip install -q pennylane
!pip install -q torch torchvision torchaudio
!pip install -q scikit-learn
!pip install -q nltk
!pip install -q tensorflow_datasets
!pip install -q -U transformers datasets
!pip install -q pandas

print("Installation Complete!")
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pennylane as qml
from pennylane import numpy as pnp

import tensorflow_datasets as tfds
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from datasets import Dataset

import os
import time
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", message=".*is_sparse is deprecated.*")


--- Phase 1: Setup and Dataset Preparation ---

Step 1: Installing Libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/6

In [4]:
# Step 2: Load IMDb Dataset
# --------------------------
print("\nStep 2: Loading IMDb Dataset...")
DATASET_LOADED = False
try:
    (ds_train, ds_test), ds_info = tfds.load(
        'imdb_reviews/subwords8k',
        split=['train[:10%]', 'test[:10%]'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
        download=True
    )
except Exception as e:
    print(f"Warning: Failed to load 'subwords8k' configuration: {e}")
    print("Attempting to load default 'plain_text' configuration...")
    try:
        (ds_train, ds_test), ds_info = tfds.load(
            'imdb_reviews',
            split=['train[:10%]', 'test[:10%]'],
            shuffle_files=True,
            as_supervised=True,
            with_info=True,
            download=True
        )
    except Exception as e2:
        print(f"Error loading IMDb dataset (default config): {e2}")

    print(f"Dataset loaded: {ds_info.name}")
    print(f"Number of training examples: {len(ds_train)}")
    print(f"Number of test examples: {len(ds_test)}")
    train_reviews = []
    train_labels = []
    test_reviews = []
    test_labels = []
    ds_train_numpy = tfds.as_numpy(ds_train)
    ds_test_numpy = tfds.as_numpy(ds_test)

    print("Converting dataset to Pandas DataFrame...")
    for review, label in ds_train_numpy:
        train_reviews.append(review.decode('utf-8', 'ignore'))
        train_labels.append(label)

    for review, label in ds_test_numpy:
        test_reviews.append(review.decode('utf-8', 'ignore'))
        test_labels.append(label)

    df_train = pd.DataFrame({'review': train_reviews, 'label': train_labels})
    df_test = pd.DataFrame({'review': test_reviews, 'label': test_labels})
    DATASET_LOADED = True

except Exception as e:
    print(f"Error loading IMDb dataset: {e}")



Step 2: Loading IMDb Dataset...


Attempting to load default 'plain_text' configuration...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.PXIEU1_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.PXIEU1_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Dataset loaded: imdb_reviews
Number of training examples: 2500
Number of test examples: 2500
Converting dataset to Pandas DataFrame...


In [5]:
# Step 3: Preprocess Text Data and Create Subset
# -----------------------------------------------
DATASET_PREPPED = False
if DATASET_LOADED:
    print("\nStep 3: Preprocessing Text and Creating Subset...")
    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords', quiet=True)
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt', quiet=True)
    try:
        nltk.data.find('tokenizers/punkt_tab')
    except LookupError:
        nltk.download('punkt_tab', quiet=True)
    stop_words_set = set(stopwords.words('english'))
    def clean_text(text):
        if not isinstance(text, str): return ""
        text = text.lower()
        text = re.sub(r'<br\s*/?>', ' ', text)
        text = re.sub(r'[^a-z\s]', '', text)
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word and word not in stop_words_set and len(word) > 1]
        return " ".join(tokens)

    print("Cleaning text data...")
    if 'review' in df_train.columns: df_train['cleaned_review'] = df_train['review'].apply(clean_text)
    else: df_train['cleaned_review'] = ""
    if 'review' in df_test.columns: df_test['cleaned_review'] = df_test['review'].apply(clean_text)
    else: df_test['cleaned_review'] = ""
    print("Text cleaning complete.")

    N_SAMPLES = 2000
    RANDOM_STATE = 42
    print(f"Creating a balanced subset of approx {N_SAMPLES} samples...")
    df_combined = pd.concat([df_train, df_test], ignore_index=True)
    df_combined.dropna(subset=['cleaned_review', 'label'], inplace=True)

    if not df_combined.empty and df_combined['label'].nunique() >= 2:
        min_class_count = df_combined['label'].value_counts().min()
        if min_class_count > 0:
            n_samples_per_class = min(N_SAMPLES // 2, min_class_count)
            if n_samples_per_class > 0:
                df_subset = df_combined.groupby('label', group_keys=False).apply(lambda x: x.sample(n=n_samples_per_class, random_state=RANDOM_STATE, replace=False))
                N_SAMPLES = len(df_subset)
                print(f"Actual subset size: {N_SAMPLES}")
                df_subset = df_subset.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

                try:
                    train_subset_df, test_subset_df = train_test_split(
                        df_subset, test_size=0.5, random_state=RANDOM_STATE, stratify=df_subset['label']
                    )
                    X_train_text_cleaned = train_subset_df['cleaned_review'].tolist()
                    y_train = train_subset_df['label'].values
                    X_test_text_cleaned = test_subset_df['cleaned_review'].tolist()
                    y_test = test_subset_df['label'].values
                    X_train_text_bert = train_subset_df['review'].tolist()
                    X_test_text_bert = test_subset_df['review'].tolist()

                    print(f"Subset created: {len(X_train_text_cleaned)} train, {len(X_test_text_cleaned)} test.")
                    DATASET_PREPPED = True
                except ValueError as e:
                     print(f"Error during train_test_split: {e}")
            else: print("Error: Cannot sample 0 items per class.")
        else: print("Error: At least one class has zero samples after cleaning.")
    else: print("Error: Combined DataFrame empty or has < 2 classes.")
else:
    print("Skipping Preprocessing: Dataset not loaded.")



Step 3: Preprocessing Text and Creating Subset...
Cleaning text data...
Text cleaning complete.
Creating a balanced subset of approx 2000 samples...
Actual subset size: 2000
Subset created: 1000 train, 1000 test.


<ipython-input-5-d7e228cec4c6>:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_subset = df_combined.groupby('label', group_keys=False).apply(lambda x: x.sample(n=n_samples_per_class, random_state=RANDOM_STATE, replace=False))


In [9]:
# Step 4: Load GloVe Embeddings (Needed for VQC/MLP and LSTM pipelines)
# --------------------------------------------------------------------
GLOVE_LOADED = False
embeddings_index = {}
if DATASET_PREPPED:
    print("\nStep 4: Loading GloVe Embeddings...")
    EMBEDDING_DIM = 100
    GLOVE_FILE = f'/content/glove.6B.{EMBEDDING_DIM}d.txt'

    if not os.path.exists(GLOVE_FILE):
        print(f"ERROR: GloVe file '{GLOVE_FILE}' not found.")
        print("Please download GloVe (e.g., glove.6B.zip), extract, and upload.")
    else:
        print(f"Loading GloVe file: {GLOVE_FILE}...")
        try:
            with open(GLOVE_FILE, encoding='utf-8') as f:
                for line in f:
                    values = line.split()
                    if len(values) < EMBEDDING_DIM + 1: continue
                    word = values[0]
                    try:
                        coefs = np.asarray(values[1:], dtype='float32')
                        if len(coefs) == EMBEDDING_DIM: embeddings_index[word] = coefs
                    except ValueError: continue
            print(f"Found {len(embeddings_index)} word vectors.")
            if len(embeddings_index) > 0: GLOVE_LOADED = True
            else: print("Warning: No embeddings loaded from file.")
        except Exception as e: print(f"Error loading GloVe: {e}")
else:
    print("Skipping GloVe loading: Dataset not prepped.")



Step 4: Loading GloVe Embeddings...
Loading GloVe file: /content/glove.6B.100d.txt...
Found 45207 word vectors.


In [10]:
# ==========================================================================================
# Pipeline 1: VQC and MLP (using GloVe Average -> PCA -> Quantum Encoding)
# ==========================================================================================
PIPELINE_1_OK = False
if DATASET_PREPPED and GLOVE_LOADED:
    print("\n\n--- Pipeline 1: VQC and MLP (GloVe -> PCA -> VQC) ---")

    # Step 5: Feature Extraction for VQC/MLP (GloVe Average -> PCA)
    # ------------------------------------------------------------
    print("\nStep 5: Feature Extraction for VQC/MLP (GloVe Average -> PCA)...")

    def get_avg_embedding(text, embedding_idx, embedding_dim):
        """Calculates the average GloVe embedding for a text."""
        tokens = text.split()
        vectors = [embedding_idx.get(word) for word in tokens if word in embedding_idx]
        if not vectors:
            return np.zeros(embedding_dim)
        return np.mean(vectors, axis=0)

    X_train_glove_avg = np.array([get_avg_embedding(text, embeddings_index, EMBEDDING_DIM) for text in X_train_text_cleaned])
    X_test_glove_avg = np.array([get_avg_embedding(text, embeddings_index, EMBEDDING_DIM) for text in X_test_text_cleaned])
    print(f"Average GloVe vectors created with shape (train): {X_train_glove_avg.shape}")

    N_QUBITS = 4
    n_pca_components = min(N_QUBITS, X_train_glove_avg.shape[0], X_train_glove_avg.shape[1])
    if n_pca_components < N_QUBITS: print(f"Warning: Reducing PCA components to {n_pca_components}.")
    if n_pca_components <= 0: print("Error: Cannot perform PCA with 0 components.")
    else:
        pca_glove = PCA(n_components=n_pca_components)
        X_train_pca_glove = pca_glove.fit_transform(X_train_glove_avg)
        X_test_pca_glove = pca_glove.transform(X_test_glove_avg)
        print(f"PCA features (from GloVe) created with shape (train): {X_train_pca_glove.shape}")
        ACTUAL_N_QUBITS = n_pca_components

        scaler_pca = StandardScaler()
        X_train_scaled_glove = scaler_pca.fit_transform(X_train_pca_glove)
        X_test_scaled_glove = scaler_pca.transform(X_test_pca_glove)
        print("PCA features scaled (StandardScaler).")

        y_train_tensor = torch.tensor(y_train).float().unsqueeze(1)
        y_test_tensor = torch.tensor(y_test).float().unsqueeze(1)
        X_train_tensor_pca_glove = torch.tensor(X_train_scaled_glove).float()
        X_test_tensor_pca_glove = torch.tensor(X_test_scaled_glove).float()

        Q_BATCH_SIZE = 8
        train_dataset_pca_glove = TensorDataset(X_train_tensor_pca_glove, y_train_tensor)
        test_dataset_pca_glove = TensorDataset(X_test_tensor_pca_glove, y_test_tensor)
        train_loader_pca_glove = DataLoader(train_dataset_pca_glove, batch_size=Q_BATCH_SIZE, shuffle=True)
        test_loader_pca_glove = DataLoader(test_dataset_pca_glove, batch_size=Q_BATCH_SIZE, shuffle=False)

        PIPELINE_1_OK = True

else:
    print("Skipping Pipeline 1 (VQC/MLP): Dataset or GloVe not ready.")

if PIPELINE_1_OK:
    # Step 6: Define Quantum Circuit (VQC) and Hybrid Model
    # ----------------------------------------------------
    print("\nStep 6: Defining Quantum Circuit and Hybrid Model...")
    N_LAYERS = 3


    dev = qml.device("default.qubit", wires=ACTUAL_N_QUBITS)

    # --- Define the VQC ---
    @qml.qnode(dev, interface='torch', diff_method='parameter-shift')
    def quantum_circuit(inputs, weights):

        qml.AngleEmbedding(inputs, wires=range(ACTUAL_N_QUBITS), rotation='Y')
        qml.StronglyEntanglingLayers(weights, wires=range(ACTUAL_N_QUBITS))
        return qml.expval(qml.PauliZ(wires=0))

    class HybridModel(nn.Module):
        def __init__(self, n_qubits, n_layers):
            super().__init__()
            self.n_qubits = n_qubits
            self.valid_model = False
            try:
                self.weights_shape = qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=n_qubits)
                initial_q_weights = torch.randn(self.weights_shape, requires_grad=True) * 0.1
                self.q_weights = nn.Parameter(initial_q_weights)
                self.valid_model = True
            except Exception as e: print(f"Error init VQC weights: {e}")

        def forward(self, features):
            if not self.valid_model: return torch.zeros((features.shape[0], 1), device=features.device)
            q_output = quantum_circuit(features, self.q_weights)
            if q_output.ndim == 1: return q_output.unsqueeze(1)
            elif q_output.ndim == 0: return q_output.unsqueeze(0).unsqueeze(1)
            else: return q_output

    model_q = HybridModel(n_qubits=ACTUAL_N_QUBITS, n_layers=N_LAYERS)
    if not model_q.valid_model: PIPELINE_1_OK = False # Stop if model failed
    else: print("Hybrid PyTorch model created.")

if PIPELINE_1_OK:
    # Step 7: Train the Quantum Model
    # -------------------------------
    print("\nStep 7: Training the Quantum Model (VQC)...")
    Q_LEARNING_RATE = 0.01
    Q_EPOCHS = 25
    device_cpu = torch.device("cpu")
    model_q.to(device_cpu)
    criterion_q = nn.BCEWithLogitsLoss()
    optimizer_q = optim.Adam(model_q.parameters(), lr=Q_LEARNING_RATE)

    print("Starting VQC Training...")
    start_time_q = time.time()
    for epoch in range(Q_EPOCHS):
        model_q.train()
        running_loss = 0.0; batch_count = 0
        for batch_inputs, batch_labels in train_loader_pca_glove:
            if batch_inputs.shape[0] == 0: continue
            batch_count += 1
            batch_inputs, batch_labels = batch_inputs.to(device_cpu), batch_labels.to(device_cpu)
            try:
                optimizer_q.zero_grad()
                outputs = model_q(batch_inputs)
                loss = criterion_q(outputs, batch_labels)
                if torch.isnan(loss): continue
                loss.backward()
                optimizer_q.step()
                running_loss += loss.item()
            except Exception as e: print(f"Err VQC train E{epoch+1}: {e}"); continue
        if batch_count > 0: print(f"VQC Epoch {epoch+1}/{Q_EPOCHS} - Avg Loss: {running_loss / batch_count:.4f}")
        else: print(f"VQC Epoch {epoch+1} - No batches"); break
    training_time_q = time.time() - start_time_q
    print(f"VQC Training Complete. Time: {training_time_q:.2f}s")

    # --- Final Evaluation for VQC ---
    model_q.eval()
    final_preds_q = []; final_true_q = []
    with torch.no_grad():
        for batch_inputs, batch_labels in test_loader_pca_glove:
             if batch_inputs.shape[0] == 0: continue
             batch_inputs, batch_labels = batch_inputs.to(device_cpu), batch_labels.to(device_cpu)
             try:
                 outputs = model_q(batch_inputs)
                 predicted = (torch.sigmoid(outputs) > 0.5).float()
                 final_preds_q.extend(predicted.cpu().numpy().flatten())
                 final_true_q.extend(batch_labels.cpu().numpy().flatten())
             except Exception as e: print(f"Err VQC eval: {e}"); continue
    if final_true_q:
        accuracy_q = accuracy_score(final_true_q, final_preds_q)
        precision_q, recall_q, f1_q, _ = precision_recall_fscore_support(final_true_q, final_preds_q, average='binary', zero_division=0)
        print("\n--- Quantum Model (VQC) Final Performance ---")
        print(f"Accuracy: {accuracy_q:.4f}, Precision: {precision_q:.4f}, Recall: {recall_q:.4f}, F1-Score: {f1_q:.4f}")
    else: print("\nVQC: No results to evaluate."); accuracy_q, precision_q, recall_q, f1_q = 0,0,0,0

if PIPELINE_1_OK:
    # Step 8: Define and Train Classical MLP Model (on same GloVe->PCA features)
    # -----------------------------------------------------------------------
    print("\nStep 8: Defining and Training Classical MLP Model...")
    class ClassicalMLP(nn.Module): # Same MLP definition as before
        def __init__(self, input_dim, hidden_dim=32):
            super().__init__()
            self.layer_1 = nn.Linear(input_dim, hidden_dim)
            self.relu = nn.ReLU()
            self.layer_2 = nn.Linear(hidden_dim, 1)
        def forward(self, x): return self.layer_2(self.relu(self.layer_1(x)))

    model_mlp = ClassicalMLP(input_dim=ACTUAL_N_QUBITS)
    criterion_mlp = nn.BCEWithLogitsLoss()
    optimizer_mlp = optim.Adam(model_mlp.parameters(), lr=Q_LEARNING_RATE)
    device_mlp = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_mlp.to(device_mlp)
    print(f"Using device: {device_mlp} for MLP training")

    # --- MLP Training Loop ---
    start_time_mlp = time.time()
    for epoch in range(Q_EPOCHS):
        model_mlp.train()
        running_loss = 0.0; batch_count = 0
        for batch_inputs, batch_labels in train_loader_pca_glove: # Use SAME loader
             if batch_inputs.shape[0] == 0: continue
             batch_count += 1
             batch_inputs, batch_labels = batch_inputs.to(device_mlp), batch_labels.to(device_mlp)
             try:
                 optimizer_mlp.zero_grad()
                 outputs = model_mlp(batch_inputs)
                 loss = criterion_mlp(outputs, batch_labels)
                 if torch.isnan(loss): continue
                 loss.backward()
                 optimizer_mlp.step()
                 running_loss += loss.item()
             except Exception as e: print(f"Err MLP train E{epoch+1}: {e}"); continue
        if batch_count > 0:
             if (epoch + 1) % 5 == 0 or epoch == Q_EPOCHS - 1: print(f"MLP Epoch {epoch+1}/{Q_EPOCHS} - Avg Loss: {running_loss / batch_count:.4f}")
        else: print(f"MLP Epoch {epoch+1} - No batches"); break
    # (End Training Loop)
    training_time_mlp = time.time() - start_time_mlp
    print(f"MLP Training Complete. Time: {training_time_mlp:.2f}s")

    # --- Final Evaluation for MLP ---
    model_mlp.eval()
    final_preds_mlp = []; final_true_mlp = []
    with torch.no_grad():
        for batch_inputs, batch_labels in test_loader_pca_glove: # Use SAME loader
             if batch_inputs.shape[0] == 0: continue
             batch_inputs, batch_labels = batch_inputs.to(device_mlp), batch_labels.to(device_mlp)
             try:
                 outputs = model_mlp(batch_inputs)
                 predicted = (torch.sigmoid(outputs) > 0.5).float()
                 final_preds_mlp.extend(predicted.cpu().numpy().flatten())
                 final_true_mlp.extend(batch_labels.cpu().numpy().flatten())
             except Exception as e: print(f"Err MLP eval: {e}"); continue
    if final_true_mlp:
        accuracy_mlp = accuracy_score(final_true_mlp, final_preds_mlp)
        precision_mlp, recall_mlp, f1_mlp, _ = precision_recall_fscore_support(final_true_mlp, final_preds_mlp, average='binary', zero_division=0)
        print("\n--- Classical MLP Model Final Performance (on GloVe->PCA features) ---")
        print(f"Accuracy: {accuracy_mlp:.4f}, Precision: {precision_mlp:.4f}, Recall: {recall_mlp:.4f}, F1-Score: {f1_mlp:.4f}")
    else: print("\nMLP: No results to evaluate."); accuracy_mlp, precision_mlp, recall_mlp, f1_mlp = 0,0,0,0




--- Pipeline 1: VQC and MLP (GloVe -> PCA -> VQC) ---

Step 5: Feature Extraction for VQC/MLP (GloVe Average -> PCA)...
Average GloVe vectors created with shape (train): (1000, 100)
PCA features (from GloVe) created with shape (train): (1000, 4)
PCA features scaled (StandardScaler).

Step 6: Defining Quantum Circuit and Hybrid Model...
Hybrid PyTorch model created.

Step 7: Training the Quantum Model (VQC)...
Starting VQC Training...
VQC Epoch 1/25 - Avg Loss: 0.6888
VQC Epoch 2/25 - Avg Loss: 0.6325
VQC Epoch 3/25 - Avg Loss: 0.6276
VQC Epoch 4/25 - Avg Loss: 0.6249
VQC Epoch 5/25 - Avg Loss: 0.6238
VQC Epoch 6/25 - Avg Loss: 0.6232
VQC Epoch 7/25 - Avg Loss: 0.6233
VQC Epoch 8/25 - Avg Loss: 0.6225
VQC Epoch 9/25 - Avg Loss: 0.6226
VQC Epoch 10/25 - Avg Loss: 0.6223
VQC Epoch 11/25 - Avg Loss: 0.6226
VQC Epoch 12/25 - Avg Loss: 0.6224
VQC Epoch 13/25 - Avg Loss: 0.6232
VQC Epoch 14/25 - Avg Loss: 0.6226
VQC Epoch 15/25 - Avg Loss: 0.6221
VQC Epoch 16/25 - Avg Loss: 0.6221
VQC Epoch

In [11]:
# ==============================================================================
# Pipeline 2: BERT Baseline (Remains the same as before)
# ==============================================================================
BERT_PIPELINE_OK = False
if DATASET_PREPPED:
    print("\n\n--- Pipeline 2: BERT Baseline ---")
    # Step 9: Setup and Train BERT Model
    print("\nStep 9: Setting up and Training BERT Model...")
    MODEL_NAME = 'bert-base-uncased'
    try:
        tokenizer_bert = BertTokenizer.from_pretrained(MODEL_NAME)
        model_bert = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
        BERT_PIPELINE_OK = True
    except Exception as e: print(f"Error loading BERT: {e}"); accuracy_bert, precision_bert, recall_bert, f1_bert, training_time_bert = 0,0,0,0,0

    if BERT_PIPELINE_OK:
        def tokenize_function_bert(examples):
            texts = [str(text) if text is not None else "" for text in examples["text"]]
            return tokenizer_bert(texts, padding="max_length", truncation=True, max_length=128)
        try:
            hf_train_dataset_bert = Dataset.from_dict({'text': X_train_text_bert, 'label': y_train})
            hf_test_dataset_bert = Dataset.from_dict({'text': X_test_text_bert, 'label': y_test})
            tokenized_train_dataset_bert = hf_train_dataset_bert.map(tokenize_function_bert, batched=True, remove_columns=["text"])
            tokenized_test_dataset_bert = hf_test_dataset_bert.map(tokenize_function_bert, batched=True, remove_columns=["text"])
            tokenized_train_dataset_bert.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
            tokenized_test_dataset_bert.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
            BERT_BATCH_SIZE = 8
            bert_train_loader = DataLoader(tokenized_train_dataset_bert, batch_size=BERT_BATCH_SIZE, shuffle=True)
            bert_test_loader = DataLoader(tokenized_test_dataset_bert, batch_size=BERT_BATCH_SIZE, shuffle=False)
            optimizer_bert = optim.AdamW(model_bert.parameters(), lr=2e-5)
            BERT_EPOCHS = 3
            total_steps_bert = len(bert_train_loader) * BERT_EPOCHS
            scheduler_bert = get_linear_schedule_with_warmup(optimizer_bert, num_warmup_steps=0, num_training_steps=total_steps_bert)
            device_bert = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model_bert.to(device_bert)
            print(f"Using device: {device_bert} for BERT training")
            print("Starting BERT Fine-Tuning...")
            start_time_bert = time.time()
            model_bert.train()
            for epoch in range(BERT_EPOCHS):
                running_loss=0.0; batch_count=0
                for batch in bert_train_loader:
                    if 'input_ids' not in batch or batch['input_ids'].shape[0]==0: continue
                    batch_count+=1
                    try:
                        optimizer_bert.zero_grad()
                        input_ids = batch['input_ids'].to(device_bert)
                        attention_mask = batch['attention_mask'].to(device_bert)
                        labels = batch['label'].to(device_bert)
                        outputs = model_bert(input_ids, attention_mask=attention_mask, labels=labels)
                        loss = outputs.loss
                        if torch.isnan(loss): continue
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)
                        optimizer_bert.step()
                        scheduler_bert.step()
                        running_loss += loss.item()
                    except Exception as e: print(f"Err BERT train E{epoch+1}: {e}"); continue
                if batch_count > 0: print(f"BERT Epoch {epoch+1}/{BERT_EPOCHS} - Avg Loss: {running_loss / batch_count:.4f}")
                else: print(f"BERT Epoch {epoch+1} - No batches"); break
            # (End Training Loop)
            training_time_bert = time.time() - start_time_bert
            print(f"BERT Fine-Tuning Complete. Time: {training_time_bert:.2f}s")
            # --- Evaluation ---
            model_bert.eval()
            final_preds_bert = []; final_true_bert = []
            with torch.no_grad():
                 for batch in bert_test_loader:
                     if 'input_ids' not in batch or batch['input_ids'].shape[0]==0: continue
                     try:
                         input_ids = batch['input_ids'].to(device_bert); attention_mask = batch['attention_mask'].to(device_bert); labels = batch['label'].to(device_bert)
                         outputs = model_bert(input_ids, attention_mask=attention_mask)
                         predictions = torch.argmax(outputs.logits, dim=-1)
                         final_preds_bert.extend(predictions.cpu().numpy().flatten())
                         final_true_bert.extend(labels.cpu().numpy().flatten())
                     except Exception as e: print(f"Err BERT eval: {e}"); continue
            if final_true_bert:
                accuracy_bert = accuracy_score(final_true_bert, final_preds_bert)
                precision_bert, recall_bert, f1_bert, _ = precision_recall_fscore_support(final_true_bert, final_preds_bert, average='binary', zero_division=0)
                print("\n--- BERT Model Final Performance ---")
                print(f"Accuracy: {accuracy_bert:.4f}, Precision: {precision_bert:.4f}, Recall: {recall_bert:.4f}, F1-Score: {f1_bert:.4f}")
            else: print("\nBERT: No results to evaluate."); accuracy_bert, precision_bert, recall_bert, f1_bert = 0,0,0,0
        except Exception as e:
            print(f"Error during BERT pipeline setup/run: {e}")
            accuracy_bert, precision_bert, recall_bert, f1_bert, training_time_bert = 0,0,0,0,0





--- Pipeline 2: BERT Baseline ---

Step 9: Setting up and Training BERT Model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Using device: cuda for BERT training
Starting BERT Fine-Tuning...
BERT Epoch 1/3 - Avg Loss: 0.5727
BERT Epoch 2/3 - Avg Loss: 0.2878
BERT Epoch 3/3 - Avg Loss: 0.1649
BERT Fine-Tuning Complete. Time: 76.19s

--- BERT Model Final Performance ---
Accuracy: 0.8510, Precision: 0.8395, Recall: 0.8680, F1-Score: 0.8535


In [12]:
# ==============================================================================
# Pipeline 3: LSTM Baseline (Remains the same as before)
# ==============================================================================
LSTM_PIPELINE_OK = False
if DATASET_PREPPED and GLOVE_LOADED:
    print("\n\n--- Pipeline 3: LSTM Baseline ---")
    # Step 10: Setup and Train LSTM Model
    print("\nStep 10: Setting up and Training LSTM Model...")
    MAX_VOCAB_SIZE = 10000
    MAX_SEQUENCE_LENGTH = 150
    try:
        print("Tokenizing text for LSTM sequences...")
        full_text_list = X_train_text_cleaned + X_test_text_cleaned
        word_counts = {}
        for text in full_text_list:
            if isinstance(text, str):
                 for word in text.split(): word_counts[word] = word_counts.get(word, 0) + 1
        sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
        word_index = {word: i + 2 for i, (word, count) in enumerate(sorted_words[:MAX_VOCAB_SIZE - 2])}
        word_index["<PAD>"] = 0; word_index["<UNK>"] = 1
        vocab_size = min(MAX_VOCAB_SIZE, len(word_index))
        print(f"LSTM Vocab size: {vocab_size}")
        def text_to_sequence(text, word_idx, max_len):
            if not isinstance(text, str): return [word_idx["<PAD>"]] * max_len
            tokens = text.split(); seq = [word_idx.get(word, word_idx["<UNK>"]) for word in tokens]
            if len(seq) > max_len: return seq[:max_len]
            else: return seq + [word_idx["<PAD>"]] * (max_len - len(seq))
        X_train_sequences = [text_to_sequence(text, word_index, MAX_SEQUENCE_LENGTH) for text in X_train_text_cleaned]
        X_test_sequences = [text_to_sequence(text, word_index, MAX_SEQUENCE_LENGTH) for text in X_test_text_cleaned]
        print("Creating embedding matrix...")
        embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
        hits = 0; misses = 0
        for word, i in word_index.items():
            if i >= vocab_size: continue
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None: embedding_matrix[i] = embedding_vector; hits += 1
            else: misses += 1
        print(f"Converted {hits} words ({misses} misses)")
        # --- Tensors & DataLoaders ---
        X_train_seq_tensor = torch.tensor(X_train_sequences, dtype=torch.long)
        X_test_seq_tensor = torch.tensor(X_test_sequences, dtype=torch.long)
        y_train_lstm_tensor = torch.tensor(y_train, dtype=torch.float).unsqueeze(1)
        y_test_lstm_tensor = torch.tensor(y_test, dtype=torch.float).unsqueeze(1)
        lstm_train_dataset = TensorDataset(X_train_seq_tensor, y_train_lstm_tensor)
        lstm_test_dataset = TensorDataset(X_test_seq_tensor, y_test_lstm_tensor)
        LSTM_BATCH_SIZE = 32
        lstm_train_loader = DataLoader(lstm_train_dataset, batch_size=LSTM_BATCH_SIZE, shuffle=True)
        lstm_test_loader = DataLoader(lstm_test_dataset, batch_size=LSTM_BATCH_SIZE, shuffle=False)

        # --- Define LSTM Model --- (Identical definition)
        class LSTMClassifier(nn.Module):
            def __init__(self, vocab_sz, embedding_d, hidden_d, output_d, embedding_mat, non_trainable=True, dropout_prob=0.3):
                super().__init__()
                self.embedding = nn.Embedding(vocab_sz, embedding_d, padding_idx=0)
                self.embedding.weight = nn.Parameter(torch.tensor(embedding_mat, dtype=torch.float))
                if non_trainable: self.embedding.weight.requires_grad = False
                self.lstm = nn.LSTM(embedding_d, hidden_d, batch_first=True, dropout=dropout_prob if dropout_prob > 0 else 0, num_layers=1)
                self.dropout = nn.Dropout(dropout_prob)
                self.fc = nn.Linear(hidden_d, output_d)
            def forward(self, text_sequences):
                embedded = self.embedding(text_sequences)
                lstm_output, (hidden, cell) = self.lstm(embedded)
                final_hidden_state = self.dropout(hidden.squeeze(0))
                return self.fc(final_hidden_state)

        # --- Instantiate & Train ---
        HIDDEN_DIM_LSTM = 64; OUTPUT_DIM_LSTM = 1; DROPOUT_LSTM = 0.3
        model_lstm = LSTMClassifier(vocab_size, EMBEDDING_DIM, HIDDEN_DIM_LSTM, OUTPUT_DIM_LSTM, embedding_matrix, dropout_prob=DROPOUT_LSTM)
        LSTM_LEARNING_RATE = 0.001; LSTM_EPOCHS = 5
        criterion_lstm = nn.BCEWithLogitsLoss()
        optimizer_lstm = optim.Adam(model_lstm.parameters(), lr=LSTM_LEARNING_RATE)
        device_lstm = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model_lstm.to(device_lstm)
        print(f"Using device: {device_lstm} for LSTM training")
        # --- Training Loop ---
        print("Starting LSTM Training...")
        start_time_lstm = time.time()
        # (LSTM Training Loop - simplified logging)
        for epoch in range(LSTM_EPOCHS):
            model_lstm.train(); running_loss=0.0; batch_count=0
            for batch_sequences, batch_labels in lstm_train_loader:
                 if batch_sequences.shape[0] == 0: continue
                 batch_count+=1
                 batch_sequences, batch_labels = batch_sequences.to(device_lstm), batch_labels.to(device_lstm)
                 try:
                     optimizer_lstm.zero_grad(); outputs = model_lstm(batch_sequences); loss = criterion_lstm(outputs, batch_labels)
                     if torch.isnan(loss): continue
                     loss.backward(); optimizer_lstm.step(); running_loss += loss.item()
                 except Exception as e: print(f"Err LSTM train E{epoch+1}: {e}"); continue
            if batch_count > 0: print(f"LSTM Epoch {epoch+1}/{LSTM_EPOCHS} - Avg Loss: {running_loss / batch_count:.4f}")
            else: print(f"LSTM Epoch {epoch+1} - No batches"); break
        # (End Training Loop)
        training_time_lstm = time.time() - start_time_lstm
        print(f"LSTM Training Complete. Time: {training_time_lstm:.2f}s")
        model_lstm.eval(); final_preds_lstm = []; final_true_lstm = []
        with torch.no_grad():
            for batch_sequences, batch_labels in lstm_test_loader:
                 if batch_sequences.shape[0] == 0: continue
                 batch_sequences, batch_labels = batch_sequences.to(device_lstm), batch_labels.to(device_lstm)
                 try:
                     outputs = model_lstm(batch_sequences); predicted = (torch.sigmoid(outputs) > 0.5).float()
                     final_preds_lstm.extend(predicted.cpu().numpy().flatten()); final_true_lstm.extend(batch_labels.cpu().numpy().flatten())
                 except Exception as e: print(f"Err LSTM eval: {e}"); continue
        if final_true_lstm:
            accuracy_lstm = accuracy_score(final_true_lstm, final_preds_lstm)
            precision_lstm, recall_lstm, f1_lstm, _ = precision_recall_fscore_support(final_true_lstm, final_preds_lstm, average='binary', zero_division=0)
            print("\n--- LSTM Model Final Performance ---")
            print(f"Accuracy: {accuracy_lstm:.4f}, Precision: {precision_lstm:.4f}, Recall: {recall_lstm:.4f}, F1-Score: {f1_lstm:.4f}")
        else: print("\nLSTM: No results to evaluate."); accuracy_lstm, precision_lstm, recall_lstm, f1_lstm = 0,0,0,0
        LSTM_PIPELINE_OK = True

    except Exception as e:
        print(f"Error during LSTM pipeline setup/run: {e}")
        accuracy_lstm, precision_lstm, recall_lstm, f1_lstm, training_time_lstm = 0,0,0,0,0

else:
    print("\n\n--- Pipeline 3: LSTM Baseline ---")
    print("SKIPPED: Dataset not prepped or GloVe not loaded.")
    accuracy_lstm, precision_lstm, recall_lstm, f1_lstm, training_time_lstm = 0,0,0,0,0





--- Pipeline 3: LSTM Baseline ---

Step 10: Setting up and Training LSTM Model...
Tokenizing text for LSTM sequences...
LSTM Vocab size: 10000
Creating embedding matrix...
Converted 8916 words (1084 misses)
Using device: cuda for LSTM training
Starting LSTM Training...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


LSTM Epoch 1/5 - Avg Loss: 0.6936
LSTM Epoch 2/5 - Avg Loss: 0.6892
LSTM Epoch 3/5 - Avg Loss: 0.6835
LSTM Epoch 4/5 - Avg Loss: 0.6745
LSTM Epoch 5/5 - Avg Loss: 0.6621
LSTM Training Complete. Time: 0.65s

--- LSTM Model Final Performance ---
Accuracy: 0.5280, Precision: 0.5155, Recall: 0.9340, F1-Score: 0.6643


In [13]:
# ==============================================================================
# Final Comparison and Analysis
# ==============================================================================
print("\n\n--- Final Comparison and Analysis ---")

results = {'VQC': {'acc': 0, 'prec': 0, 'rec': 0, 'f1': 0, 'time': 0},
           'MLP': {'acc': 0, 'prec': 0, 'rec': 0, 'f1': 0, 'time': 0},
           'LSTM': {'acc': 0, 'prec': 0, 'rec': 0, 'f1': 0, 'time': 0},
           'BERT': {'acc': 0, 'prec': 0, 'rec': 0, 'f1': 0, 'time': 0}}

if 'accuracy_q' in locals(): results['VQC'] = {'acc': accuracy_q, 'prec': precision_q, 'rec': recall_q, 'f1': f1_q, 'time': training_time_q}
if 'accuracy_mlp' in locals(): results['MLP'] = {'acc': accuracy_mlp, 'prec': precision_mlp, 'rec': recall_mlp, 'f1': f1_mlp, 'time': training_time_mlp}
if 'accuracy_lstm' in locals(): results['LSTM'] = {'acc': accuracy_lstm, 'prec': precision_lstm, 'rec': recall_lstm, 'f1': f1_lstm, 'time': training_time_lstm}
if 'accuracy_bert' in locals(): results['BERT'] = {'acc': accuracy_bert, 'prec': precision_bert, 'rec': recall_bert, 'f1': f1_bert, 'time': training_time_bert}

print("\n--- Performance Comparison ---")
print(f"{'Model':<15} | {'Accuracy':<10} | {'Precision':<10} | {'Recall':<10} | {'F1-Score':<10} | {'Train Time (s)':<15}")
print("-" * 80)
print(f"{'Quantum VQC':<15} | {results['VQC']['acc']:<10.4f} | {results['VQC']['prec']:<10.4f} | {results['VQC']['rec']:<10.4f} | {results['VQC']['f1']:<10.4f} | {results['VQC']['time']:<15.2f}")
print(f"{'Classical MLP':<15} | {results['MLP']['acc']:<10.4f} | {results['MLP']['prec']:<10.4f} | {results['MLP']['rec']:<10.4f} | {results['MLP']['f1']:<10.4f} | {results['MLP']['time']:<15.2f}")
print(f"{'LSTM (GloVe)':<15} | {results['LSTM']['acc']:<10.4f} | {results['LSTM']['prec']:<10.4f} | {results['LSTM']['rec']:<10.4f} | {results['LSTM']['f1']:<10.4f} | {results['LSTM']['time']:<15.2f}")
print(f"{'BERT':<15} | {results['BERT']['acc']:<10.4f} | {results['BERT']['prec']:<10.4f} | {results['BERT']['rec']:<10.4f} | {results['BERT']['f1']:<10.4f} | {results['BERT']['time']:<15.2f}")
print("-" * 80)

print("\n--- Analysis ---")
print("Key Observations:")
if DATASET_PREPPED:
     print(f"1. Dataset Size: Small subset ({N_SAMPLES} samples).")
     print(f"2. Input Features:")
     if PIPELINE_1_OK: print(f"   - VQC & MLP used {ACTUAL_N_QUBITS}-dim PCA of averaged GloVe embeddings.")
     if LSTM_PIPELINE_OK: print(f"   - LSTM used sequences of {EMBEDDING_DIM}-dim GloVe word embeddings.")
     if BERT_PIPELINE_OK: print(f"   - BERT used its own tokenization/embeddings.")
else: print("1. Dataset preparation failed.")

print(f"3. Training Time: VQC simulation is often slow. Classical models vary.")
print("4. Performance:")
f1_scores = {model: data['f1'] for model, data in results.items() if data['f1'] > 0}
if f1_scores: best_model = max(f1_scores, key=f1_scores.get); print(f"  - Best F1-score on this run: {best_model} (F1: {f1_scores[best_model]:.4f})")
else: print("  - No models completed successfully.")
if PIPELINE_1_OK and results['MLP']['f1'] > 0: print(f"  - VQC vs MLP (same input): VQC F1={results['VQC']['f1']:.4f}, MLP F1={results['MLP']['f1']:.4f}")




--- Final Comparison and Analysis ---

--- Performance Comparison ---
Model           | Accuracy   | Precision  | Recall     | F1-Score   | Train Time (s) 
--------------------------------------------------------------------------------
Quantum VQC     | 0.6610     | 0.6903     | 0.5840     | 0.6327     | 1216.59        
Classical MLP   | 0.6730     | 0.6909     | 0.6260     | 0.6569     | 4.89           
LSTM (GloVe)    | 0.5280     | 0.5155     | 0.9340     | 0.6643     | 0.65           
BERT            | 0.8510     | 0.8395     | 0.8680     | 0.8535     | 76.19          
--------------------------------------------------------------------------------

--- Analysis ---
Key Observations:
1. Dataset Size: Small subset (2000 samples).
2. Input Features:
   - VQC & MLP used 4-dim PCA of averaged GloVe embeddings.
   - LSTM used sequences of 100-dim GloVe word embeddings.
   - BERT used its own tokenization/embeddings.
3. Training Time: VQC simulation is often slow. Classical models var